In [1]:
import sklearn

In [7]:
from sklearn.datasets import make_circles
n_samples = 1000

X,y = make_circles(n_samples,noise=0.03,random_state=42)

In [9]:
import pandas as pd
circles = pd.DataFrame({
    'X':X[:0],
    'Y':X[:1],
    
})
circles

ValueError: Per-column arrays must each be 1-dimensional